<a href="https://colab.research.google.com/github/kim-jason/PowerliftingDataset/blob/main/Powerlifting_Data_Science_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Powerlifting Dataset EDA

Name: Jason Kim  <br>
Email: jason.s.kim20@gmail.com <br>
Linkedin: https://www.linkedin.com/in/jasonks/

## Introduction

### Overview of notebook


The purpose of this notebook is to analyze a dataset consisting of powerlifting meets in order to identify trends and insights by age, country, federation, and other provided and generated features. <br>

We will be using the CSV dataset from [OpenPowerlifting](https://openpowerlifting.gitlab.io/opl-csv/introduction.html), an open source site that regularly compiles official results from powerlifting competitions around the world. <br>

### Overview of powerlifting

Powerlifting is a sport consisting of three specific lifts: squat, bench press, and deadlift. The sum of the best lift in each discipline determines the winner in a competition. Lifters compete in given age groups and weight categories. 

 <!-- [![Everything Is AWESOME](http://i.imgur.com/Ot5DWAW.png)](https://youtu.be/StTqXEQ2l-Y?t=35s "Everything Is AWESOME") -->

 TO DO: Add description fo three


## Data import

In [1]:
# Import libraries used in the notebook
import pandas as pd
from datetime import datetime, timedelta
import math

In [2]:
# Download the raw CSV dataset (note I saved this CSV file to my personal Google Drive as it makes it faster to download and use in Google Colab)
! gdown --id 1b15_F2lH2sHI02m7F1HGQkDw3Bl25-X9

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1b15_F2lH2sHI02m7F1HGQkDw3Bl25-X9
To: /content/openpowerlifting-2022-08-06-98216f16.csv
100% 540M/540M [00:03<00:00, 160MB/s]


In [3]:
# Create dataframe with the raw CSV dataset
df = pd.read_csv('openpowerlifting-2022-08-06-98216f16.csv')
df.head()
print(len(df))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (33,35,38) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


2755243


In [4]:
# Describe the contents of the dataframe
df.describe().apply(lambda s: s.apply('{0:.1f}'.format))

,Age,BodyweightKg,Squat1Kg,Squat2Kg,Squat3Kg,Squat4Kg,Best3SquatKg,Bench1Kg,Bench2Kg,Bench3Kg,...,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,Best3DeadliftKg,TotalKg,Dots,Wilks,Glossbrenner,Goodlift
count,1720150.0,2719385.0,710550.0,702587.0,682372.0,7313.0,1821444.0,1158272.0,1144372.0,1107394.0,...,810608.0,795214.0,759616.0,19399.0,1970884.0,2564651.0,2541034.0,2541034.0,2541034.0,2318067.0
mean,30.8,84.1,113.5,95.8,36.6,75.7,174.3,83.5,56.3,-16.5,...,159.6,131.8,20.7,80.1,189.1,375.4,273.0,272.0,256.5,63.9
std,13.2,22.7,140.2,164.7,193.0,182.8,69.0,101.3,126.2,141.0,...,108.3,157.4,211.4,187.3,62.2,206.0,129.6,129.1,122.7,16.3
min,0.0,10.0,-555.0,-580.0,-600.5,-550.0,-508.0,-515.0,-589.7,-575.0,...,-461.0,-502.5,-587.5,-500.0,-410.0,1.0,0.7,0.7,0.6,0.5
25%,20.5,67.1,90.0,75.0,-155.0,-88.0,122.5,55.0,-45.0,-135.0,...,125.0,115.0,-202.5,-75.5,140.0,200.0,141.6,141.0,133.5,52.2
50%,27.5,81.9,145.0,145.0,115.0,131.0,170.0,105.0,95.0,-60.0,...,180.0,177.5,120.0,142.9,188.2,351.5,294.6,293.6,274.1,63.6
75%,38.5,98.8,200.0,202.5,192.5,198.5,217.7,145.0,145.0,117.5,...,224.5,230.0,205.0,206.4,235.0,530.7,371.1,369.5,350.0,75.2
max,98.0,300.0,555.0,577.5,595.0,592.4,595.0,551.0,500.0,577.5,...,450.0,467.5,487.5,440.5,487.5,1407.5,795.2,793.3,756.9,167.5


In [5]:
# Confirm datatypes of the features
df.dtypes

Name                 object
Sex                  object
Event                object
Equipment            object
Age                 float64
AgeClass             object
BirthYearClass       object
Division             object
BodyweightKg        float64
WeightClassKg        object
Squat1Kg            float64
Squat2Kg            float64
Squat3Kg            float64
Squat4Kg            float64
Best3SquatKg        float64
Bench1Kg            float64
Bench2Kg            float64
Bench3Kg            float64
Bench4Kg            float64
Best3BenchKg        float64
Deadlift1Kg         float64
Deadlift2Kg         float64
Deadlift3Kg         float64
Deadlift4Kg         float64
Best3DeadliftKg     float64
TotalKg             float64
Place                object
Dots                float64
Wilks               float64
Glossbrenner        float64
Goodlift            float64
Tested               object
Country              object
State                object
Federation           object
ParentFederation    

In [6]:
# Update Date from object to datetime
df['Date'] = pd.to_datetime(df['Date'])
df.dtypes

Name                        object
Sex                         object
Event                       object
Equipment                   object
Age                        float64
AgeClass                    object
BirthYearClass              object
Division                    object
BodyweightKg               float64
WeightClassKg               object
Squat1Kg                   float64
Squat2Kg                   float64
Squat3Kg                   float64
Squat4Kg                   float64
Best3SquatKg               float64
Bench1Kg                   float64
Bench2Kg                   float64
Bench3Kg                   float64
Bench4Kg                   float64
Best3BenchKg               float64
Deadlift1Kg                float64
Deadlift2Kg                float64
Deadlift3Kg                float64
Deadlift4Kg                float64
Best3DeadliftKg            float64
TotalKg                    float64
Place                       object
Dots                       float64
Wilks               

Here is a description of each feature in the provided dataset. Additional details for each feature can be found [here](https://openpowerlifting.gitlab.io/opl-csv/bulk-csv-docs.html). 

*   **Name**: Name of the lifter
*   **Sex**: Sex category in which the lifter competed (M, F, or Mx)
*   **Event**: The type of competition that the lifter entered (SBD, BD, SD, SB, S, B, D)
*   **Equipment**: Equipment category under which the lifts were performed (Raw, Wraps, Single-ply, Multi-ply, Unlimited, Straps)
*   **Age**: Age of the lifter on the start date of the meet
*   **AgeClass**: Age class in which the filter falls, based on exact age of the lifter on the day of the competition
*   **BirthYearClass**: Birth year class in which the filter falls
*   **Division**: Division of competition
*   **BodyweightKg**: Recorded bodyweight of the lifter at the time of competition
*   **WeightClassKg**: Weight class in which the lifter competed
*   **Squat1Kg, Bench1Kg, Deadlift1Kg**: First attempts for each of squat, bench, and deadlift, respectively
*   **Squat2Kg, Bench2Kg, Deadlift2Kg**: Second attempts for each of squat, bench, and deadlift, respectively
*   **Squat3Kg, Bench3Kg, Deadlift3Kg**: Third attempts for each of squat, bench, and deadlift, respectively
*   **Squat4Kg, Bench4Kg, Deadlift4Kg**: Fourth attempts for each of squat, bench, and deadlift, respectively
*   **Best3SquatKg, Best3BenchKg, Best3DeadliftKg**: Maximum of the first three successful attempts for the lift
*   **TotalKg**: Sum of Best3SquatKg, Best3BenchKg, and Best3DeadliftKg, if all three lifts were a success
*   **Place**: Recorded place of the lifter in the given division at the end of the meet
*   **Dots**: Dots points (formula used to assess strength of lifter)
*   **Wilks**: Wilks points (formula used to assess strength of lifter)
*   **Glossbrenner**: Glossbrenner points (formula used to assess strength of lifter)
*   **Goodlift**: IPF GL points (formula used to assess strength of lifter)
*   **Tested**: Yes if the lifter entered a drug-tested category, empty otherwise
*   **Country**: Home country of the lifter
*   **State**: Home state/province/oblast/division/etc of the lifter
*   **Federation**: The federation that hosted the meet
*   **ParentFederation**: Topmost federation that sanctioned the meet, usually the international body
*   **Date**: Start date of the meet
*   **MeetCountry**: Country in which the meet was held
*   **MeetState**: State, province, or region in which the meet was held
*   **MeetName**: Name of the meet

## Data cleaning

### Missing data


We will go through the different features to handle missing data.

In [7]:
# See the prevalence of missing data within our current features
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

display(missing_data)

,Total,Percent
Squat4Kg,2747930,0.997346
Bench4Kg,2736202,0.993089
Deadlift4Kg,2735844,0.992959
State,2245365,0.814943
Squat3Kg,2072871,0.752337
Squat2Kg,2052656,0.745000
Squat1Kg,2044693,0.742110
Deadlift3Kg,1995627,0.724302
Deadlift2Kg,1960029,0.711382
Deadlift1Kg,1944635,0.705794


It is expected that there will be missing data for several features, such as the bench, squat, and deadlift features (some competitions do not include all three lifts), as well as the state and country features. 

One feature that seems fairly high in missing data is age. We will try to clean and impute data for this feature. This feature is important for several analyses, including progression over time by lifters, maximum lifts by age, etc. 

We will first change age data that lists the age as less than 14 to null. Firstly, according to the [2022 technical rulebook](https://www.powerlifting.sport/rules/codes/info/technical-rules) of the International Powerlifting Federation, the largest powerlifting federation in the world, the minimum age is 14. Thus, any ages below that threshold can be considered incorrect. Secondly, although other federations permit "youth" or "junior" categories for those under 14, given the purpose of this dataset is to provide insights for adults and because there are physical differences between youths and adults, I will filter out those under 14.

In [8]:
# Change any ages that are considered incorrect to null
df.loc[df.Age < 14, ['Age']] = None

# Visualize the missing rows for age
null_age_df = df[df['Age'].isnull()]
display(null_age_df.head(5))

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
76,Tatyana Labuzova,F,D,Raw,NaN,5-12,NaN,Y,64.7,65,...,Yes,Russia,NaN,GFP,NaN,2019-10-05,Russia,ORL,Oryol,Fortior est ferrum Tournament
266,Rosimeire Santos Souza,F,B,Raw,NaN,45-49,NaN,F 45-49,56.0,56,...,NaN,NaN,NaN,CONBRAP,GPC,2017-05-13,Brazil,NaN,Anhembi,Campeonato Paulista de Supino e Lev.Terra
267,João Vitor Rodrigues Claro,M,B,Raw,NaN,13-15,NaN,MT 13-15,56.0,56,...,NaN,NaN,NaN,CONBRAP,GPC,2017-05-13,Brazil,NaN,Anhembi,Campeonato Paulista de Supino e Lev.Terra
269,Eduardo Haggar da Silva,M,B,Raw,NaN,13-15,NaN,MT 13-15,60.0,60,...,NaN,NaN,NaN,CONBRAP,GPC,2017-05-13,Brazil,NaN,Anhembi,Campeonato Paulista de Supino e Lev.Terra
270,Murieu Moreira Moreno,M,B,Raw,NaN,13-15,NaN,MT 13-15,75.0,75,...,NaN,NaN,NaN,CONBRAP,GPC,2017-05-13,Brazil,NaN,Anhembi,Campeonato Paulista de Supino e Lev.Terra


We will attempt to impute missing data entries for age in two methods:


1.   Use a lifter's age from a different competition entry to calculate their age for other competitions (more accurate)
2.   Use the average of their AgeClass range or their BirthYearClass range (less accurate)



In [9]:
# Use a lifter's age from a different competition entry to calculate their age for other competitions 
lifters_missing_age = df[df['Age'].isnull()][['Name', 'Date']]
lifters_missing_age = lifters_missing_age.rename(columns={'Date':'Date_missing'})
lifters_missing_age = lifters_missing_age.drop_duplicates(subset=['Name', 'Date_missing'])

# Sort original dataset to find a row with an existing Age and Date to calculate birthday
df = df.sort_values(['Age', 'Date'])
lifters_missing_age = pd.merge(lifters_missing_age, df[['Name', 'Age', 'Date']].drop_duplicates(subset=['Name']), on='Name', how='inner')
lifters_missing_age = lifters_missing_age.rename(columns={'Age':'Age_existing', 'Date':'Date_existing'})
lifters_missing_age = lifters_missing_age[(lifters_missing_age['Age_existing'].notnull()) & (lifters_missing_age['Date_existing'].notnull())]
lifters_missing_age['Age_missing'] = lifters_missing_age['Date_missing'] - (lifters_missing_age['Date_existing'] - lifters_missing_age['Age_existing'].apply(lambda x: timedelta(seconds=x*(60*60*24*365))))
lifters_missing_age['Age_missing'] = lifters_missing_age['Age_missing'].apply(lambda x: round((x.total_seconds() / 60 / 60 / 24 / 365.25)*2)/2) # We round to the nearest 0.5 increment as the original dataset did this

display(lifters_missing_age)

,Name,Date_missing,Age_existing,Date_existing,Age_missing
28,Serafim Rocha,2017-05-13,49.5,1998-10-06,68.0
29,Serafim Rocha,2018-04-14,49.5,1998-10-06,69.0
30,Serafim Rocha,2017-07-16,49.5,1998-10-06,68.0
31,Serafim Rocha,2018-03-11,49.5,1998-10-06,69.0
32,Serafim Rocha,2018-10-21,49.5,1998-10-06,69.5
...,...,...,...,...,...
1005961,Dmitriy Dvornikov,2004-08-09,14.5,2018-05-12,0.5
1005968,Sergey Sosnovskiy,2004-08-09,17.5,2001-06-01,20.5
1006111,Erik Setnický,2009-07-01,15.5,2011-10-22,13.0
1006112,Johan Smith,2005-09-07,14.0,2006-09-05,13.0


In [10]:
# Create a copy of the original dataframe 
cleaned_df = df.copy(deep=True)

# Merge with the original dataframe to add in the estimated ages
cleaned_df = pd.merge(cleaned_df, lifters_missing_age[['Name', 'Date_missing', 'Age_missing']], how='left', left_on=['Name', 'Date'], right_on=['Name', 'Date_missing'])
cleaned_df.loc[cleaned_df.Age.isnull() & cleaned_df.Date_missing.notnull() & cleaned_df.Age_missing.notnull(), ['Age']] = cleaned_df['Age_missing']
cleaned_df = cleaned_df.drop(['Date_missing', 'Age_missing'], axis=1)

# Use the average of the lifters's AgeClass range or the BirthYearClass range to estimate additional missing ages
def get_average_age(x):
  if math.isnan(x['Age']):
    if len(str(x['AgeClass'])) > 0 and '-' in str(x['AgeClass']):
      min, max = x['AgeClass'].split('-')
      average = (float(min) + float(max)) / 2.0
      return round(average * 2) / 2
    elif len(str(x['BirthYearClass'])) > 0 and '-' in str(x['BirthYearClass']):
      min, max = x['BirthYearClass'].split('-')
      average = (float(min) + float(max)) / 2.0
      return round(average * 2) / 2
    else:
      return x['Age']
  else:
    return x['Age']
cleaned_df['Age'] = cleaned_df.apply(get_average_age, axis=1)

display(cleaned_df)

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
0,R. Coleman,M,SBD,Single-ply,14.0,13-15,14-18,Open,60.0,60,...,NaN,NaN,NaN,USPF,IPF,1979-12-01,USA,WY,Casper,Holiday Festival Tournament
1,Ed Bridges,M,SBD,Single-ply,14.0,13-15,14-18,Open,56.0,56,...,NaN,USA,NaN,USPF,IPF,1979-12-01,USA,IA,NaN,Annual Mid-West Open
2,H. Rodriguez,M,SBD,Single-ply,14.0,13-15,14-18,Open,90.0,90,...,NaN,NaN,NaN,USPF,IPF,1979-12-08,USA,WI,Milwaukee,Badger State Open
3,Todd Sorenson,M,SBD,Single-ply,14.0,13-15,14-18,Open,52.0,52,...,NaN,NaN,NaN,USPF,IPF,1980-01-19,USA,ND,Fargo,Red River Open
4,R. Coleman,M,SBD,Single-ply,14.0,13-15,14-18,High School,82.5,82.5,...,NaN,NaN,NaN,LHSPLA,NaN,1980-01-19,USA,LA,Ruston,Louisiana Tech High School Meet
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755238,Coach Abbey,M,BD,Raw,NaN,NaN,NaN,Open,100.0,100,...,Yes,Uganda,NaN,UgandaPA,WPA,2022-07-30,Uganda,NaN,Jinja,Jinja Fitness Festival
2755239,Martin Jacoby,M,B,Raw,NaN,NaN,NaN,Open,84.0,90,...,Yes,Uganda,NaN,UgandaPA,WPA,2022-07-30,Uganda,NaN,Jinja,Jinja Fitness Festival
2755240,Eric Ayude,M,D,Raw,NaN,NaN,NaN,Open,77.0,82.5,...,Yes,Uganda,NaN,UgandaPA,WPA,2022-07-30,Uganda,NaN,Jinja,Jinja Fitness Festival
2755241,Enock Sparks,M,D,Raw,NaN,NaN,NaN,Open,100.0,100,...,Yes,NaN,NaN,UgandaPA,WPA,2022-07-30,Uganda,NaN,Jinja,Jinja Fitness Festival


We can see that there are some estimated ages that are less than 14, which again is the minimum age to compete. This suggests that either the entries with missing ages had incorrect dates or that the entries with existing ages and dates are incorrect. We will assume the former and as the 36,299 rows is only 1% of the total dataset of 2,755,243 rows, we will simply remove these rows from the original dataset. Note that this cleaned dataset will be used for analyses involving age and dates, but we can use the original dataset for other analyses that don't require accurate ages and dates.

We can confirm all data entries now have recorded or estimated ages.

In [11]:
# Remove data entries with estimated or recorded ages below the minimum threshold of 14
display(cleaned_df[cleaned_df['Age'] < 14])
cleaned_df = cleaned_df[cleaned_df['Age'] >= 14]

# Confirm all remaining data entries have recorded or imputed ages
display(cleaned_df[cleaned_df['Age'].isnull()])

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
1693703,Ruben Melendez,M,SBD,Wraps,-34.5,NaN,NaN,Open,56.00,56,...,NaN,USA,NaN,AAU,NaN,1964-09-05,USA,PA,York,The Powerlifting Tournament Of America
1693736,Ruben Melendez,M,SBD,Raw,-33.5,NaN,NaN,Open,56.00,56,...,NaN,USA,NaN,AAU,NaN,1965-09-04,USA,PA,York,Senior Nationals
1693761,William Collins,M,SBD,Wraps,-40.0,NaN,NaN,Open,90.00,90,...,NaN,USA,NaN,AAU,NaN,1965-09-04,USA,PA,York,Senior Nationals
1693766,Robert Weaver,M,SBD,Wraps,-29.0,NaN,NaN,Open,122.47,90+,...,NaN,NaN,NaN,AAU,NaN,1965-09-04,USA,PA,York,Senior Nationals
1693782,R. Jackson,M,SBD,Wraps,6.5,NaN,NaN,Open,82.50,82.5,...,Yes,NaN,NaN,BAWLA,IPF,1966-02-27,UK,NaN,NaN,British Men's Powerlifting Championships
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755086,Oliver Salonen,M,B,Raw,8.5,5-12,NaN,M_YR_WPC,86.30,90,...,NaN,Finland,NaN,WPC-Finland,WPC,2022-07-23,Finland,NaN,Savonlinna,Summer Powerlifting Party
2755109,Cameron Brown #6,M,D,Raw,8.5,5-12,NaN,Open,50.80,52,...,NaN,USA,FL,USPA,IPL,2022-07-23,USA,FL,Ft. Walton Beach,Summer Slam
2755227,Amelia Benvenuto,F,SBD,Raw,8.5,5-12,NaN,FR-Y2-G,56.60,60,...,Yes,USA,NJ,USAPL,NaN,2022-07-30,USA,PA,NaN,Pennsylvania State Championship
2755228,Lawson Benvenuto,M,SBD,Raw,8.5,5-12,NaN,MR-Y2-G,49.80,52,...,Yes,USA,NJ,USAPL,NaN,2022-07-30,USA,PA,NaN,Pennsylvania State Championship


,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName


We will also remove estimated ages above 100 as it can be safely assumed those are not realistic ages for competitive powerlifters. Online research also suggests that the age 100 is an appropriate threshold, as [Edith Traina](https://barbend.com/100-year-old-edith-murway-traina-powerlifter-guinness/) was recently entered into the 2022 Guinness Book of Records as the oldest competitive female powerlifter ever. We can see that her entries are included below.

Below we can see several of the rows in which the age estimations are to be removed. For example, Gerald Foster is estimated to be 539.5 years old, as this data was imputed by taking the average of his AgeClass of 80-999. 

In [23]:
cleaned_df = cleaned_df.sort_values(by='Age', ascending=False)
display(cleaned_df[cleaned_df['Age'] > 90])

display(cleaned_df[cleaned_df['Name'] == 'Edith Traina'])

cleaned_df = cleaned_df[cleaned_df['Age'] >= 100]

,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
2596713,Gerald Foster,M,BD,Raw,539.5,80-999,70-999,Masters 85-89,67.50,67.5,...,Yes,USA,FL,WABDL,NaN,2019-11-12,USA,NV,Las Vegas,World Championships
2048526,George Choi,M,D,Single-ply,539.5,80-999,70-999,Masters 80-84,82.50,82.5,...,Yes,USA,CA,WABDL,NaN,2010-11-16,USA,NV,Las Vegas,World Championships
1958371,Edwin Free Jr,M,D,Single-ply,539.5,80-999,70-999,Masters 80-84,90.00,90,...,Yes,USA,NaN,WABDL,NaN,2004-05-12,USA,AL,Gadsden,Riverfest BP/DL
2246994,Hobert Peake,M,SBD,Wraps,539.5,80-999,70-999,Grand-Masters,116.94,118,...,NaN,USA,AL,SPF,NaN,2015-01-24,USA,AL,NaN,North Alabama Classic
2082193,Helen Herring,F,D,Single-ply,539.5,80-999,70-999,Masters 80-84,56.00,56,...,Yes,USA,AL,WABDL,NaN,2011-07-09,USA,TN,Nashville,National Championships
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2519812,John Weiss,M,B,Raw,90.5,55-59,NaN,Masters 55-59,89.09,90,...,NaN,USA,NaN,SPF,NaN,2018-11-03,USA,TN,Gatlinburg,Record Breakers
1693625,Dmytro Krastelev,M,B,Raw,90.5,80-999,70-999,Pro Open,112.50,125,...,NaN,Ukraine,NaN,NAP,IPA,2017-04-08,Ukraine,NaN,Inkerman,Inkerman Cup
1693623,Evelyn Kilgore,F,D,Single-ply,90.5,80-999,70-999,Masters 90,82.50,82.5,...,Yes,USA,CA,WABDL,NaN,2016-03-05,USA,CA,Chico,CA State BP & DL
1693629,Sam Penner,M,B,Raw,90.5,80-999,70-999,MR-M6,75.83,83,...,Yes,USA,SC,USAPL,IPF,2021-06-14,USA,FL,Daytona Beach,Raw Bench Nationals


,Name,Sex,Event,Equipment,Age,AgeClass,BirthYearClass,Division,BodyweightKg,WeightClassKg,...,Tested,Country,State,Federation,ParentFederation,Date,MeetCountry,MeetState,MeetTown,MeetName
1693701,Edith Traina,F,B,Raw,98.0,80-999,70-999,Open,67.50,67.5,...,NaN,NaN,NaN,RAWU,NaN,2019-11-10,USA,FL,Tampa,Tony Conyers Extravaganza
1693695,Edith Traina,F,BD,Raw,97.0,80-999,70-999,SNR,75.00,75,...,NaN,NaN,NaN,RAWU,NaN,2018-09-15,USA,FL,Tampa,Push/Pull
1693692,Edith Traina,F,D,Raw,96.0,80-999,70-999,SNR,75.00,75,...,NaN,NaN,NaN,RAWU,NaN,2018-06-08,USA,FL,Tampa,Grip it and Rip it Deadlift Contest
1693691,Edith Traina,F,B,Raw,96.0,80-999,70-999,95-99,67.13,67.5,...,NaN,NaN,NaN,RAWU,NaN,2017-12-30,USA,FL,Mount Dora,Beasty Bash
2369359,Edith Traina,F,BD,Raw,94.5,80-999,70-999,90-94,75.00,75,...,NaN,NaN,NaN,RAWU,NaN,2016-09-23,USA,FL,Tampa,Tony Conyers Extravaganza
2346028,Edith Traina,F,BD,Raw,94.0,80-999,70-999,90-94,75.00,75,...,NaN,NaN,NaN,RAWU,NaN,2016-02-21,USA,FL,Tampa,Mike Witmer Memorial
2292907,Edith Traina,F,SBD,Raw,93.5,NaN,NaN,SNR,75.00,75,...,NaN,NaN,NaN,RAWU,NaN,2015-07-18,USA,FL,Port St. Lucie,Senior Powerlifting Championships
2299403,Edith Traina,F,BD,Raw,93.5,80-999,70-999,90-94,75.00,75,...,NaN,NaN,NaN,RAWU,NaN,2015-11-05,USA,FL,Tampa,Tony Conyers Extravaganza
1693664,Edith Traina,F,BD,Raw,93.0,80-999,70-999,90-94,75.00,75,...,NaN,NaN,NaN,RAWU,NaN,2015-01-24,USA,FL,Tampa,Mike Witmer Memorial
2286697,Edith Traina,F,SBD,Raw,93.0,80-999,70-999,90-94,75.00,75,...,NaN,NaN,NaN,RAWU,NaN,2015-04-18,USA,FL,Melbourne,Teenage & Masters Championships
